In [28]:
import gevent

In [29]:
def foo():
    print('[1] Running in foo')
    gevent.sleep(0)
    print('[2] Emplict context switch to foo again')

def bar():
    print('[3] Emplict context to bar')
    gevent.sleep(0)
    print('[4] implicit switch switch back to bar')

gevent.joinall([
    gevent.spawn(foo),
    gevent.spawn(bar),
])

[1] Running in foo
[3] Emplict context to bar
[2] Emplict context switch to foo again
[4] implicit switch switch back to bar


[<Greenlet at 0x7fb6c8ab7ef0: _run>, <Greenlet at 0x7fb6b86e9710: _run>]

## Synchronous & Asynchronous Execution

In [30]:
import time
from gevent import select

In [31]:
# gevent의 진짜 힘은 상호작용 하며 스케쥴링 될 수 있는 네트워크와 IO bound 함수들을 작성할 때 발휘된다.
# gevent는 네트워크 라이브러리들이 암시적으로 greenlet 컨텍스트들이 가능한 시점에 암시적으로 yield 하도록 보장합니다 (==> 뭔 소리인지...)

start = time.time()
tic = lambda: 'at %1.1f seconds' % (time.time() - start)

def gr1():
    # Busy waits for a second, but we don't want to stick around...
    print('[1] Started Polling: %s' % tic())
    select.select([], [], [], 2)
    print('[2] Ended Polling: %s' % tic())

def gr2():
    # Busy waits for a second, but we don't want to stick around...
    print('[3] Started Polling: %s' % tic())
    select.select([], [], [], 2)
    print('[4] Ended Polling: %s' % tic())

def gr3():
    print("[5] Hey lets do some stuff while the greenlets poll, %s" % tic())
    gevent.sleep(1)

gevent.joinall([
    gevent.spawn(gr1),
    gevent.spawn(gr2),
    gevent.spawn(gr3),
])

[1] Started Polling: at 0.0 seconds
[3] Started Polling: at 0.0 seconds
[5] Hey lets do some stuff while the greenlets poll, at 0.0 seconds
[2] Ended Polling: at 2.0 seconds
[4] Ended Polling: at 2.0 seconds


[<Greenlet at 0x7fb6b86e9950: _run>,
 <Greenlet at 0x7fb6b86e9b90: _run>,
 <Greenlet at 0x7fb6b86e9a70: _run>]

> `.spawn(?)`에 들어가면 coroutine에 들어간다고 보면 될 거 같다..

In [32]:
import random

def task(pid):
    """
    Some non-deterministic task
    """
    gevent.sleep(random.randint(0, 2) * 0.001)
    print("Task %s done" % pid)

def synchronous():
    for i in range(1, 10):
        task(i)

def asynchronous():
    threads = [gevent.spawn(task, i) for i in range(10)]
    gevent.joinall(threads)

print('Synchronous:')
synchronous()

print('Asynchronous:')
asynchronous()

Synchronous:
Task 1 done
Task 2 done
Task 3 done
Task 4 done
Task 5 done
Task 6 done
Task 7 done
Task 8 done
Task 9 done
Asynchronous:
Task 4 done
Task 6 done
Task 7 done
Task 8 done
Task 9 done
Task 0 done
Task 1 done
Task 2 done
Task 3 done
Task 5 done


> 동기(synchronous)처리 시 모든 task들이 순차적으로 실행되고, 다른 task들이 각각 동작하는 동한 blocking 방식으로 동작한다. <br/>
비동기(asynchronous)처리 시 실행 순서가 보장되지 않고 실행시간이 동기처리 시 보다 훨씬 줄어든다. task들이 서로  실행을 block 하지 않기에

In [33]:
import gevent.monkey
gevent.monkey.patch_socket()

import urllib.request
import simplejson as json

def fetch(pid):
    response = urllib.request.urlopen('http://worldtimeapi.org/api/timezone/America/Argentina/Salta')
    result = response.read()
    json_result = json.loads(result)
    datetime = json_result['datetime']
    
    print('Process %s: %s' % (pid, datetime))
    return json_result['datetime']

def synchronous():
    for i in range(1, 10):
        fetch(i)

def asynchronous():
    threads = []
    for i in range(1, 10):
        threads.append(gevent.spawn(fetch, i))
    gevent.joinall(threads)

print('Synchronous:')
synchronous()

print('Asynchronous:')
asynchronous()

Synchronous:
Process 1: 2020-12-11T02:09:39.563529-03:00
Process 2: 2020-12-11T02:09:40.110631-03:00
Process 3: 2020-12-11T02:09:40.661135-03:00
Process 4: 2020-12-11T02:09:41.218996-03:00
Process 5: 2020-12-11T02:09:41.782401-03:00
Process 6: 2020-12-11T02:09:42.333535-03:00
Process 7: 2020-12-11T02:09:42.885888-03:00
Process 8: 2020-12-11T02:09:43.441464-03:00
Process 9: 2020-12-11T02:09:43.997512-03:00
Asynchronous:
Process 9: 2020-12-11T02:09:44.551578-03:00
Process 4: 2020-12-11T02:09:44.553950-03:00
Process 1: 2020-12-11T02:09:44.554538-03:00
Process 2: 2020-12-11T02:09:44.555124-03:00
Process 8: 2020-12-11T02:09:44.561080-03:00
Process 5: 2020-12-11T02:09:44.560272-03:00
Process 6: 2020-12-11T02:09:44.560763-03:00
Process 7: 2020-12-11T02:09:44.558524-03:00
Process 3: 2020-12-11T02:09:44.563216-03:00


> 위의 예시로, sync와 async의 차이를 확연히 볼 수 있다.

## Determinism

In [34]:
import time

def echo(i):
    time.sleep(0.001)
    return i

# Non Deterministic Process Pool
from multiprocessing.pool import Pool

p = Pool(10)
run1 = [a for a in p.imap_unordered(echo, range(10))]
run2 = [a for a in p.imap_unordered(echo, range(10))]
run3 = [a for a in p.imap_unordered(echo, range(10))]
run4 = [a for a in p.imap_unordered(echo, range(10))]

print("Non Deterministic Process Pool: ", run1 == run2 == run3 == run4)

# Deterministic Gevent Pool
from gevent.pool import Pool

p = Pool(10)
run1 = [a for a in p.imap_unordered(echo, range(10))]
run2 = [a for a in p.imap_unordered(echo, range(10))]
run3 = [a for a in p.imap_unordered(echo, range(10))]
run4 = [a for a in p.imap_unordered(echo, range(10))]

print("Deterministic Gevent Pool: ", run1 == run2 == run3 == run4)


Non Deterministic Process Pool:  False
Deterministic Gevent Pool:  True


> gevent가 일반적으로 dterministic 하다고 해도, 소켓과 파일과 같은 외부 서비스와 연동할 때 non-deterministic한 입력들이 들어올 수 있다.
동시성을 다룰 때 만날 수 있는 문제로 `race condition`이 있는데, 이를 피하기 위해서는 공유 자원을 사용하지 않는 것이다.

## Spawning Greenlets

In [35]:
import gevent
from gevent import Greenlet

def foo(message, n):
    """
    Each thread will be passed the message, and n arguments
    in its initialization.
    """
    gevent.sleep(n)
    print(message)

# Initialize a new Greenlet instance running the named function 'foo'
thread1 = Greenlet.spawn(foo, "Hello", 1)

# Wrapper for creating and running a new Greenlet from the named function 'foo', with the passed arguments
thread2 = gevent.spawn(foo, "I live!", 2)

# Lambda expressions
thread3 = gevent.spawn(lambda x: (x+1), 2)

threads = [thread1, thread2, thread3]

gevent.joinall(threads)


Hello
I live!


[<Greenlet at 0x7fb6b86e9cb0: _run>,
 <Greenlet at 0x7fb6b86e9290: _run>,
 <Greenlet at 0x7fb6b86e9170: _run>]

> 위에서 계쏙 해왔던 것들인데_ 왜 지금와서야 이걸 알려주는지...

In [36]:
class MyGreenlet(Greenlet):
    def __init__(self, message, n):
        Greenlet.__init__(self)
        self.message = message
        self.n = n
    
    def _run(self):
        print(self.message)
        gevent.sleep(self.n)

g = MyGreenlet("Hi there!", 3)
g.start()
g.join()

Hi there!


> Greenlet 클래스를 오버라이드 해서 쓸 수 있다는 것을 다시 보여주고 있음

 ## Program shutdown

In [37]:
import gevent
from gevent import Timeout

seconds = 3

timeout = Timeout(seconds)
timeout.start()

def wait():
    gevent.sleep(3)

try:
    gevent.spawn(wait).join()
except Timeout:
    print('Could not complete')


Could not complete


## Monkeypatching

In [38]:
import socket
print(socket.socket)

print("[a] After monkey patch")
from gevent import monkey
monkey.patch_socket()
print("[1] ", socket.socket)

import select
print("[2] ", select.select)
monkey.patch_select()
print("[b] After monkey patch")
print("[3] ", select.select)

<class 'gevent._socket3.socket'>
[a] After monkey patch
[1]  <class 'gevent._socket3.socket'>
[2]  <function select at 0x7fb6c89a35f0>
[b] After monkey patch
[3]  <function select at 0x7fb6c89a35f0>


> Python's runtime allows for most objects to be modified at runtime including modules, classes, and even functions. This is generally an astoudingly bad idea since it creates an "implicit side-effect" that is most often extremely difficult to debug if problems occur, nevertheless in extreme situations where a library needs to alter the fundamental behavior of Python itself monkey patches can be used. In this case gevent is capable of patching most of the blocking system calls in the standard library including those in socket, ssl, threading and select modules to instead behave cooperatively.

For example, the Redis python bindings normally uses regular tcp sockets to communicate with the redis-server instance. Simply by invoking gevent.monkey.patch_all() we can make the redis bindings schedule requests cooperatively and work with the rest of our gevent stack.

This lets us integrate libraries that would not normally work with gevent without ever writing a single line of code. While monkey-patching is still evil, in this case it is a "useful evil".


> 위에거... 뭔말인지 이해가 가지는 않음...

## Events

In [39]:
import gevent
from gevent.event import Event

'''
Illustrates the use of events
'''

evt = Event()

def setter():
    '''After 3 seconds, wake all threads waiting on the value of evt'''
    print('[1] A: Hey wait for me, I have to do something')
    gevent.sleep(3)
    print("[2] Ok, I'm done")
    evt.set()

def waiter():
    '''After 3 seconds the get all will unblock'''
    print("[3] I'll wait for you")
    evt.wait()
    print("[4] It's about time")

def main():
    gevent.joinall([
        gevent.spawn(setter),
        gevent.spawn(waiter),
        gevent.spawn(waiter),
        gevent.spawn(waiter),
        gevent.spawn(waiter),
    ])

if __name__ == '__main__': main()

[1] A: Hey wait for me, I have to do something
[3] I'll wait for you
[3] I'll wait for you
[3] I'll wait for you
[3] I'll wait for you
[2] Ok, I'm done
[4] It's about time
[4] It's about time
[4] It's about time
[4] It's about time


## Queues

In [44]:
import gevent
from gevent.queue import Queue

tasks = Queue()

def worker(n):
    while not tasks.empty():
        task = tasks.get()
        print("[1] Worker %s got task %s" % (n, task))
        gevent.sleep(0)
    
    print('[2] Quitting time!')

def boss():
    for i in range(1, 25):
        tasks.put_nowait(i)

gevent.spawn(boss).join()

gevent.joinall([
    gevent.spawn(worker, 'steve'),
    gevent.spawn(worker, 'john'),
    gevent.spawn(worker, 'nancy'),
])

[1] Worker steve got task 1
[1] Worker john got task 2
[1] Worker nancy got task 3
[1] Worker steve got task 4
[1] Worker john got task 5
[1] Worker nancy got task 6
[1] Worker steve got task 7
[1] Worker john got task 8
[1] Worker nancy got task 9
[1] Worker steve got task 10
[1] Worker john got task 11
[1] Worker nancy got task 12
[1] Worker steve got task 13
[1] Worker john got task 14
[1] Worker nancy got task 15
[1] Worker steve got task 16
[1] Worker john got task 17
[1] Worker nancy got task 18
[1] Worker steve got task 19
[1] Worker john got task 20
[1] Worker nancy got task 21
[1] Worker steve got task 22
[1] Worker john got task 23
[1] Worker nancy got task 24
[2] Quitting time!
[2] Quitting time!
[2] Quitting time!


[<Greenlet at 0x7fb6b86f33b0: _run>,
 <Greenlet at 0x7fb6b86f34d0: _run>,
 <Greenlet at 0x7fb6b86f3710: _run>]

> 